In [103]:
%run set_up.py
%run lit_utility_functions_2025.ipynb

#csv table where references' title, journal, year and DOI are written
reflist_csv = Path(datdir, 'test_list.csv')
#csv table with 200 sampled articles
reflist_csv_sample = Path(datdir, 'test_list_sample.csv')

In [ ]:
#Import openalex records
oalex_records_pkl = regex_list_files(in_dir=resdir, 
                       in_pattern='oalex_records_.*'
                      )[-1]

with open(oalex_records_pkl, 'rb') as f:
    oalex_records_dict = pickle.load(f)

oalex_records_pd = pd.concat(
    [
        list_of_dicts_to_dataframe(
            list_of_dicts=search_records, 
            keys_to_keep=['id', 'doi', 'title']
        ) 
        for search_records in oalex_records_dict.values()
    ]
)
#print(oalex_records_pd)

In [78]:
#Get titles and DOIs from test list
testlist_title_dois = get_testlist(
    library_id='4842799', 
    api_key_path=Path('zotero_key.txt'),
    test_list_name='test list')
#print(testlist_title_dois)

In [128]:
check = list(testlist_title_dois.values())[0]
print(re.sub("[^a-zA-Z\d\s]", "", check[0].lower()))
print(f'https://doi.org/{check[1]}')
#set(oalex_records_pd.title.str.lower())

assessing streamflow characteristics as limiting factors on benthic invertebrate assemblages in streams across the western united states
https://doi.org/10.1111/j.1365-2427.2008.02024.x


In [136]:
#~~~~~~~~~~~~~~ Check which items in test list were retrieved through the search ~~~~~~~
returned = {}
not_returned = {}

for k, v in testlist_title_dois.items():
    if ((re.sub("[^a-zA-Z\d\s]", "", v[0].lower()) 
         in set(oalex_records_pd.title.str.lower()))
        or (f'https://doi.org/{v[1]}' in set(oalex_records_pd.doi))):
        returned[k] = v[0]
    else:
        not_returned[k] = v[0]

{'GSGR3DDN': 'Responses of Native and Nonnative Fishes to Natural Flow Regime Mimicry in the San Juan River', 'IP646MXV': 'Fish assemblages in subtropical rivers: low-flow hydrology dominates hydro-ecological relationships', 'TR45GP8C': 'Using abiotic drivers of fish spawning to inform environmental flow management', 'LABLENVA': 'Linking river flow regimes to riparian plant guilds: A community-wide modeling approach', '2HSSV9YE': 'Ecohydrological and socioeconomic integration for the operational management of environmental flows', 'N57I6QJF': 'Assessment of environmental flows under limited data availability: case study of the Acheloos River, Greece'}


In [ ]:
##################### NOT USED ANYMORE/YET/TOCHECK ##############################

In [ ]:
###########CHECK: FOR WOS search
#Read and compile references from the WoS search
#into a single document set (lit_study format)
reflist = rpickle_bibdocset(
    in_dirpath=scoping_4_datdir, 
    in_pattern=".*savedrecs(\([0-9]{1,2}\))*[.]bib", 
    out_pickle=scoping_4_docset_pickle
)

#Write basic metadata from reference list to csv ~~~~~~~~~~~~~~~~
#Get all dois and titles in references returned from search
reflist_pd = tabulate_searchlist(in_reflist=reflist, 
                                 out_csvpath=reflist_csv) 

print(len(set(reflist_pd.title)))

if not reflist_csv_sample.exists():
    reflist_pd_sample = reflist_pd.sample(100)
    reflist_pd_sample.to_csv(reflist_csv_sample)
else: 
    reflist_pd_sample = pd.read_csv(reflist_csv_sample)

#Compute the number of included and excluded references depending on all catgeories for each document
#Format as a table to be examined
wos_initlist_catsample_format = pd.read_csv(catsamplecsv_path)

wos_catsample_stats = pd.melt(frame=wos_initlist_catsample_format,
        id_vars=['Unnamed: 0', 'Inclusion'],
        value_vars = wos_cat_split.columns,
        #var_name='wos_catnum',
        value_name = 'wos_catnum'
        )

wos_catsample_stats['wos_catnum'] = wos_catsample_stats['wos_catnum'].replace(r"^ +| +$", r"", regex=True)

wos_catsample_stats = wos_catsample_stats.groupby(['wos_catnum', 'Inclusion']).size().to_frame('N').reset_index().\
    pivot(index='wos_catnum', columns='Inclusion')

wos_catsample_stats.columns = wos_catsample_stats.columns.droplevel()
wos_catsample_stats.reset_index(inplace=True)
wos_catsample_stats = wos_catsample_stats.rename(columns={0:"excluded", 1:"included"}).\
    fillna(0)
wos_catsample_stats['N'] = wos_catsample_stats['excluded'] + wos_catsample_stats['included']
wos_catsample_stats.sort_values(['included','N'], ascending=[False, False]).\
to_csv(Path(scopdir, 'wos_search1_catsample_stats.csv'))